In [1]:
library(extrafont)
library(tidyverse)
library(epidemia)
library(arrow)

Registering fonts with R

── Attaching core tidyverse packages ───────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ─────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘arrow’


The following object is masked from ‘package:lubridate’:

    duration


The following object is masked from ‘package:utils’:

    timestamp




# 1. parameters

In [2]:
T = 60
T0 = 40

## observation parameters

In [3]:
alpha = 0.01
r_Y = 10

$$Y_t = NegBinom( \frac{EY_t}{r_Y + EY_t}, r_Y )$$

$$ EY_t = \alpha\sum_{s=1}^{t-1} I_s \pi_{t-s} $$

## latent parameters

In [5]:
d = 2
beta = c(0.75, -2.95)

In [6]:
r_I = 100
K = 6.5

$$I_t = NegBinom( \frac{EI_t}{r_I + EI_t}, r_I )$$

$$ EI_t = R(\bar{A}_t, \beta) \sum_{s=1}^{t-1} I_s g_{t-s} $$

$$R(\bar{A}_t, \beta) = \frac{K}{1+\exp(- \beta^\top A_t)}$$

## seed values

In [7]:
mu = log(100)

$$I_{-T_0} = \mu_0$$

# 2. fit

In [8]:
num_sim = 1000

In [9]:
data <- read_feather('data/simulation_randinf_3.feather')

In [10]:
head(data)

sim,date,R,EI,infection,EY,death,intervention
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
0,0,4.414662,441.4631,480,0.9560691,1,0
0,1,4.414662,472.2075,475,0.9625449,0,0
0,2,4.414662,583.5170,588,0.9683382,1,0
0,3,4.414662,762.1575,670,0.9740151,1,0
0,4,4.414662,997.2407,955,0.9812028,0,0
0,5,4.414662,1291.2474,1355,0.9923307,0,0


In [11]:
rt <- epirt(formula = R(country, date) ~ 1 + intervention,
            prior = shifted_gamma(shape = 1/6, scale = 1, shift = log(1.05)/6),
            prior_covariance = decov(shape = c(2, rep(0.5, 5)), scale = 0.25),
            link = scaled_logit(6.5))

In [12]:
inf <- epiinf(gen = EuropeCovid$si, seed_days = T0)

In [13]:
deaths <- epiobs(formula = death ~ 1, i2o = EuropeCovid2$inf2death,
                 prior_intercept = normal(0, 0.2), link = scaled_logit(0.02))

Warning message:
“i2o does not sum to one. Please ensure this is intentional.”


In [14]:
results = array(0, c(num_sim, 6))
colnames(results)=c('b[0]','b[1]','V[0,0]','V[1,0]','V[0,1]','V[1,1]')

In [15]:
for(iter_sim in 1:num_sim){
    start_sim = proc.time()[3]
    
    data_raw = data %>% filter(sim == iter_sim-1)
    data_i = data.frame(country = factor(rep(1,T+T0)),
                        date = as.Date("2022-01-01")
                               + c(-T0:-1, data_raw$date),
                        death = c(rep(NA,T0), data_raw$death),
                        intervention = c(rep(0,T0), data_raw$intervention))
    
    args <- list(rt = rt, inf = inf, obs = deaths, data = data_i, 
                 seed=12345, refresh=0)
    args$algorithm <- "fullrank"; args$iter <- 5e4; args$tol_rel_obj <- 1e-3
    
    fm <- do.call(epim, args)
    
    results[iter_sim,1:2] = colMeans(as.data.frame(fm$stanfit)[1:2])
    results[iter_sim,3:6] = cov(as.data.frame(fm$stanfit)[1:2])
    
    print(paste0(iter_sim,"-th simulation finished, ",
                 proc.time()[3] - start_sim," sec."))
    flush(stdout())
}

Warning message:
“Pareto k diagnostic value is 0.93. Resampling is unreliable. Increasing the number of draws or decreasing tol_rel_obj may help.”


[1] "1-th simulation finished, 6.1339999999982 sec."
[1] "2-th simulation finished, 6.33099999999831 sec."
[1] "3-th simulation finished, 4.24199999999837 sec."
[1] "4-th simulation finished, 6.00500000000102 sec."
[1] "5-th simulation finished, 6.0669999999991 sec."
[1] "6-th simulation finished, 6.05999999999767 sec."


Warning message:
“Pareto k diagnostic value is 0.79. Resampling is unreliable. Increasing the number of draws or decreasing tol_rel_obj may help.”


[1] "7-th simulation finished, 6.43600000000151 sec."
[1] "8-th simulation finished, 2.61200000000099 sec."
[1] "9-th simulation finished, 2.96700000000055 sec."
[1] "10-th simulation finished, 2.77299999999741 sec."
[1] "11-th simulation finished, 6.60699999999997 sec."
[1] "12-th simulation finished, 4.40200000000186 sec."
[1] "13-th simulation finished, 2.84899999999834 sec."


Warning message:
“Pareto k diagnostic value is 0.89. Resampling is unreliable. Increasing the number of draws or decreasing tol_rel_obj may help.”


...

[1] "995-th simulation finished, 5.18699999999808 sec."


Warning message:
“Pareto k diagnostic value is 0.86. Resampling is unreliable. Increasing the number of draws or decreasing tol_rel_obj may help.”


[1] "996-th simulation finished, 5.14400000000023 sec."
[1] "997-th simulation finished, 3.60800000000017 sec."


Warning message:
“Pareto k diagnostic value is 0.93. Resampling is unreliable. Increasing the number of draws or decreasing tol_rel_obj may help.”


[1] "998-th simulation finished, 5.21399999999994 sec."
[1] "999-th simulation finished, 5.15099999999802 sec."
[1] "1000-th simulation finished, 5.0940000000046 sec."


In [16]:
write_feather(as.data.frame(results), 
              'coverage_epidemia_nbinom_randinf_3.feather')